In [45]:
import sagemaker
import boto3
from sagemaker.estimator import Estimator
from sagemaker import image_uris, get_execution_role
from sagemaker.session import s3_input, Session

In [27]:
import os
from sagemaker.inputs import TrainingInput

In [21]:
bucket_name = 'bankapplication-bucket112233'           #  unique name for bucket
my_region = boto3.session.Session().region_name     # to set the region of the instance
print(my_region)

us-east-1


In [22]:
session = boto3.session.Session(region_name=my_region)

# Bind client to us-east-1 (optionally force the global endpoint)
s3 = session.client(
    "s3",
    region_name="us-east-1",
    endpoint_url="https://s3.amazonaws.com",   # <-- ensures us-east-1 global endpoint
)

# to verify where the client is actually pointing
print("session region:", session.region_name)
print("client region :", s3.meta.region_name)
print("AWS_DEFAULT_REGION:", os.environ.get("AWS_DEFAULT_REGION"))
print("AWS_REGION:", os.environ.get("AWS_REGION"))

try:
    # For us-east-1 you MUST NOT send CreateBucketConfiguration
    s3.create_bucket(Bucket=bucket_name)
    print("S3 bucket created successfully")
except Exception as e:
    print("S3 error:", e)

session region: us-east-1
client region : us-east-1
AWS_DEFAULT_REGION: None
AWS_REGION: None
S3 bucket created successfully


In [23]:
# set an output path where the trained model will be saved
prefix = 'xgboost-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://bankapplication-bucket112233/xgboost-built-in-algo/output


### Downloading The Dataset And Storing in S3

In [24]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [25]:
### Train Test split

import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)
     

(28831, 61) (12357, 61)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [29]:
### Saving Train And Test Into Buckets
## We start with Train Data

pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = TrainingInput(
    s3_data=f's3://{bucket_name}/{prefix}/train',
    content_type='text/csv'
)

In [30]:
# Test Data Into Buckets
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = TrainingInput(
    s3_data=f's3://{bucket_name}/{prefix}/test',
    content_type='text/csv'
)

#### Building Models Xgboot- Inbuilt Algorithm

In [40]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
container = image_uris.retrieve(
                framework="xgboost",
                region=my_region,
                 version="1.7-1"   # pick a supported version, e.g. "1.7-1"
                    )

In [41]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [46]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = Estimator(
    image_uri=container,                   
    hyperparameters=hyperparameters,
    role=get_execution_role(),
    instance_count=1,                        
    instance_type="ml.m5.2xlarge",           
    volume_size=5,                           
    output_path=output_path,
    use_spot_instances=True,                 
    max_run=300,                            
    max_wait=600                            
)

In [47]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2025-09-19-15-51-45-332


2025-09-19 15:51:46 Starting - Starting the training job...
2025-09-19 15:52:20 Downloading - Downloading input data...
2025-09-19 15:52:35 Downloading - Downloading the training image......
2025-09-19 15:53:21 Training - Training image download completed. Training in progress../miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
[2025-09-19 15:53:44.201 ip-10-0-121-132.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-09-19 15:53:44.267 ip-10-0-121-132.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2025-09-19:15:53:44:INFO] Imported framework sagemaker_xgboost_container.training
[2025-09-19:15:53:44:INFO] Failed to parse hyperparameter objective

#### Deploy Machine Learning Model As Endpoints

In [48]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2025-09-19-15-54-32-324
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2025-09-19-15-54-32-324
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2025-09-19-15-54-32-324


-------!

#### Prediction of the Test Data

In [49]:
from sagemaker.predictor import csv_serializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 from sagemaker.predictor import csv_serializer                                               │
│   2 test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into a     │
│   3 xgb_predictor.content_type = 'text/csv' # set the data type for an inference                 │
│   4 xgb_predictor.serializer = csv_serializer # set the serializer type                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ImportError: cannot import name 'csv_serializer' from 'sagemaker.predictor' 
(/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/predictor.py)

In [ ]:
predictions_array

In [ ]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))

#### Deleting The Endpoints

In [13]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)                                  │
│ ❱ 2 bucket_to_delete.objects.all().delete()                                                      │
│   3                                                                                              │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/boto3/resources/collection.py │
│ :555 in batch_action                                                                             │
│                                                                                                  │
│   552 │   │   action = BatchAction(action_model)                                                 │
│   553 │   │                                                                                      │
│   554 │   │   def batch_action(self, *args, **kwargs):                                           │
│ ❱ 555 │   │   │   return action(self, *args, **kwargs)                                           │
│   556 │   │                                                                                      │
│   557 │   │   batch_action.__name__ = str(snake_cased)                                           │
│   558 │   │   batch_action.__doc__ = docstring.BatchActionDocstring(                             │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/boto3/resources/action.py:134 │
│ in __call__                                                                                      │
│                                                                                                  │
│   131 │   │   # Unlike the simple action above, a batch action must operate                      │
│   132 │   │   # on batches (or pages) of items. So we get each page, construct                   │
│   133 │   │   # the necessary parameters and call the batch operation.                           │
│ ❱ 134 │   │   for page in parent.pages():                                                        │
│   135 │   │   │   params = {}                                                                    │
│   136 │   │   │   for index, resource in enumerate(page):                                        │
│   137 │   │   │   │   # There is no public interface to get a service name                       │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/boto3/resources/collection.py │
│ :169 in pages                                                                                    │
│                                                                                                  │
│   166 │   │   # Now that we have a page iterator or single page of results                       │
│   167 │   │   # we start processing and yielding individual items.                               │
│   168 │   │   count = 0                                                                          │
│ ❱ 169 │   │   for page in pages:                                                                 │
│   170 │   │   │   page_items = []                                                                │
│   171 │   │   │   for item in self._handler(self._parent, params, page):                         │
│   172 │   │   │   │   page_items.append(item)                                                    │
│                                                            

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 print(s3.meta.region_name)                                                                   │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'ResourceMeta' object has no attribute 'region_name'